In [1]:
from __future__ import print_function
from IPython.display import display
from ipywidgets import *
from traitlets import *

import numpy as np
import pandas as pd
import bqplot as bq

In [2]:
from bqplot import *

In [3]:
from bqplot import pyplot as plt

In [4]:
from traitlets import dlink

In [5]:
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)

In [6]:
from ipywidgets import Label, ToggleButtons, VBox

In [7]:

# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [8]:
def hvel(r,dph,bh,rmaxh,kh):
    x=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    return (bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**x

In [9]:
bh=.81
vmaxh=41.15
kh=.04
rmaxh=21090.
dph=4350.
pch=pn-dph

In [10]:
rh=np.linspace(0.01,maxR,100)

### read TC data

In [11]:
tc=pd.read_csv('tmp/inpData.txt',delimiter='\t')

In [12]:
tc

,time,lat,long,vmax,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,0.0,14.2,-65.5,60,0.0,0.0,0.0,0.0,73.0,0.0,69.0,0.0,151.0,53.0,118.0,146.0
1,1.0,14.2,-65.7,57,0.0,0.0,0.0,0.0,66.0,35.0,56.0,0.0,156.0,52.0,101.0,154.0
2,2.0,14.3,-65.9,66,35.0,0.0,0.0,0.0,62.0,0.0,55.0,42.0,142.0,50.0,114.0,136.0
3,3.0,14.3,-66.1,65,0.0,0.0,29.0,0.0,61.0,28.0,59.0,0.0,137.0,43.0,130.0,150.0
4,4.0,14.3,-66.3,66,0.0,27.0,38.0,0.0,66.0,33.0,79.0,0.0,144.0,44.0,160.0,144.0
5,5.0,14.3,-66.4,61,0.0,0.0,0.0,0.0,51.0,37.0,78.0,28.0,137.0,178.0,182.0,146.0
6,6.0,14.4,-66.6,61,0.0,0.0,0.0,0.0,56.0,38.0,78.0,37.0,136.0,45.0,144.0,143.0
7,7.0,14.4,-66.9,61,0.0,0.0,0.0,0.0,65.0,25.0,69.0,0.0,139.0,56.0,156.0,122.0
8,8.0,14.5,-67.1,63,0.0,0.0,0.0,0.0,67.0,28.0,61.0,37.0,135.0,48.0,148.0,110.0
9,9.0,14.6,-67.3,64,0.0,0.0,25.0,0.0,59.0,33.0,64.0,38.0,129.0,41.0,156.0,73.0


In [13]:
wind_radii=np.array([34.,50.,64.])*kt2ms

p64=tc.loc[7][['64ne','64se', '64sw', '64nw']].astype(float)

p50=tc.loc[7][['50ne','50se', '50sw', '50nw']].astype(float)

p34=tc.loc[7][['34ne','34se', '34sw', '34nw']].astype(float)

vmax=tc.loc[7].vmax*kt2ms

dp=dph#tc.loc[7].dp

x_data=np.concatenate([p34.values,p50.values,p64.values])

x_data[x_data==0.]=np.nan

y_data=np.concatenate([np.ones(4)*wind_radii[0],np.ones(4)*wind_radii[1],np.ones(4)*wind_radii[2]])

y_data[np.isnan(x_data)]=np.nan

y_data=y_data[~np.isnan(y_data)]

x_data=x_data[~np.isnan(x_data)]*nm2m

In [14]:
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

ax_x = bq.Axis(label='X Axis', scale=x_sc, grid_lines='solid')
ax_y = bq.Axis(label='Y Axis', scale=y_sc, orientation='vertical', grid_lines='solid')

Line = bq.Lines(x=rh,
                     y=hvel(rh, dph, bh, rmaxh, kh),
                     animate_dur=1000,
                     scales={'x': x_sc, 'y': y_sc})

LineVmax = bq.Lines(x=rh,
                     y=np.ones(rh.size)*vmax,
                     animate_dur=1000,
                     scales={'x': x_sc, 'y': y_sc},colors=['black'], line_style = 'dashed' )

#rmse_label = bq.Label(x=[0.75], y=[0.10], font_size=52, font_weight='bolder', colors=['orange'],
#                   text=[str(rmse.sum())], enable_move=True)


scatter_chart = Scatter(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc},default_colors=['red'])
scatter_chart.enable_move=True


slider1 = FloatSlider(value=0., min=10.e2 , max=200.e2 , step=100., description='Adjust dph')
slider2 = FloatSlider(value=0.5, min=0, max=2.5, step=0.01, description='Adjust B')
slider3 = FloatSlider(value=20000, min=1000, max=200000, step=100, description='Adjust Rmax')
slider4 = FloatSlider(value=0., min=0., max=.15, step=.01, description='Adjust kh')





# Wiring the function parameter with a slider
def handle_dph(change):
    new = change['new']
    Line.y = hvel(rh, new, slider2.value, slider3.value, slider4.value)

handle_dph({'new': dph})
slider1.observe(handle_dph, names=['value'])


def handle_bh(change):
    new = change['new']
    Line.y = hvel(rh, slider1.value, new, slider3.value, slider4.value)

handle_bh({'new': 0.5})
slider2.observe(handle_bh, names=['value'])

def handle_rmaxh(change):
    new = change['new']
    Line.y = hvel(rh, slider1.value, slider2.value, new, slider4.value)

handle_rmaxh({'new': rmaxh})
slider3.observe(handle_rmaxh, names=['value'])


def handle_kh(change):
    new = change['new']
    Line.y = hvel(rh, slider1.value, slider2.value, slider3.value, new )

handle_kh({'new': kh})
slider4.observe(handle_kh, names=['value'])

db_index = Label(color='Black', font_size='24px', value='[]')
index_sel = IndexSelector(scale=x_sc, marks=[Line])

## Now, we define a function that will be called when the selectors are interacted with
def index_change_callback(change):
    db_index.value = 'The selected radius is ' + str(change.new)

index_sel.observe(index_change_callback, names=['selected'])


fig = bq.Figure(axes=[ax_x, ax_y], marks=[Line, scatter_chart, LineVmax], title_style={'font-size': '20px','fill': 'DarkOrange'}, title='',interaction=index_sel)


#compute rmse
#rmse=hvel(x_data, slider1.value, slider2.value, slider3.value, slider4.value)-y_data
#rmse_label.text = [str(rmse.sum())]
    

VBox([fig, db_index, slider1, slider2, slider3, slider4])

